# CMPO For Finite Temperature

Some key point: ﬁnite temperature using continuous matrix product operator representation; without incurring any time discretization error; direct access to physical observables; verifying the method using the prototypical quantum XXZ chains; apply it to quantum Ising models with power-law decaying interactions and on the inﬁnite cylinder.

In [7]:
using LinearAlgebra
using Zygote

function eigensolver(M)
    # Manually symmetrize M before the eigen decomposition
    M_sym = 0.5 * (M + M')
    return eigen(M_sym)
end

function logtrexpm(beta, mat)
    # Eigensolver
    F = eigen(mat)
    w = F.values
    v = F.vectors
    y = logsumexp(beta * w)
    scaled_rho = beta * v * Diagonal(exp.(beta * w - y)) * v'
    return y, scaled_rho
end

function logsumexp(x)
    c = maximum(x)
    return c + log(sum(exp.(x .- c)))
end

logsumexp (generic function with 1 method)

In [11]:
using LinearAlgebra

struct CMPO
    Q::Array{Float64, 2}
    L::Array{Float64, 3}
    R::Array{Float64, 3}
    P::Array{Float64, 4}
    dim::Int

    function CMPO(Q, L, R, P)
        new(Q, L, R, P, size(Q, 1))
    end
end

function detach(cmpo::CMPO)
    Q, L, R, P = deepcopy(cmpo.Q), deepcopy(cmpo.L), deepcopy(cmpo.R), deepcopy(cmpo.P)
    return CMPO(Q, L, R, P)
end

function project(cmpo::CMPO, U::Array{Float64, 2})
    Q = U' * cmpo.Q * U

    L = mapslices(x -> U' * x * U, cmpo.L, [1, 2])
    R = mapslices(x -> U' * x * U, cmpo.R, [1, 2])
    P = mapslices(x -> U' * x * U, cmpo.P, [1, 2])

    return CMPO(Q, L, R, P)
end

function transpose_cmpo(cmpo::CMPO)
    Q = cmpo.Q
    L = cmpo.R
    R = cmpo.L
    P = permutedims(cmpo.P, (2, 1, 4, 3))
    return CMPO(Q, L, R, P)
end

# Usage example
Q = rand(2, 2)
L = rand(2, 2, 2)
R = rand(2, 2, 2)
P = rand(2, 2, 2, 2)

cmpo_instance = CMPO(Q, L, R, P)
detached_cmpo = detach(cmpo_instance)
U = rand(2, 2)
projected_cmpo = project(cmpo_instance, U)
transposed_cmpo = transpose_cmpo(cmpo_instance)


LoadError: MethodError: no method matching mapslices(::var"#13#16"{Matrix{Float64}}, ::Array{Float64, 3}, ::Vector{Int64})

[0mClosest candidates are:
[0m  mapslices(::Any, ::AbstractArray; dims)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:3153[24m[39m


In [12]:
using LinearAlgebra
using TensorOperations

struct CMPS
    Q::Array{Float64, 2}
    R::Array{Float64, 3}
    dim::Int

    function CMPS(Q, R)
        new(Q, R, size(Q, 1))
    end
end

function detach(cmps::CMPS)
    Q, R = deepcopy(cmps.Q), deepcopy(cmps.R)
    return CMPS(Q, R)
end

function project(cmps::CMPS, U::Array{Float64, 2})
    Q = U' * cmps.Q * U
    R = [U' * cmps.R[:, :, i] * U for i in 1:size(cmps.R, 3)]
    R = cat(R..., dims=3)
    return CMPS(Q, R)
end

function diagQ(cmps::CMPS)
    F = eigen(cmps.Q)
    U = F.vectors
    return project(cmps, U)
end

diagQ (generic function with 1 method)

In [16]:
using LinearAlgebra

struct CMPS
    Q::Array{Float64, 2}
    R::Array{Float64, 3}
    dim::Int

    function CMPS(Q, R)
        new(Q, R, size(Q, 1))
    end
end

struct CMPO
    Q::Array{Float64, 2}
    L::Array{Float64, 3}
    R::Array{Float64, 3}
    P::Array{Float64, 4}
    dim::Int

    function CMPO(Q, L, R, P)
        new(Q, L, R, P, size(Q, 1))
    end
end

function multiply(W::Array{Float64, 2}, mps::CMPS)
    R1 = Array{Float64, 3}(undef, size(mps.R))
    for i in 1:size(mps.R, 3)
        R1[:, :, i] = W * mps.R[:, :, i]
    end
    return CMPS(mps.Q, R1)
end

function act(mpo::CMPO, mps::CMPS)
    Do, Ds = mpo.dim, mps.dim
    d = size(mps.R, 1)
    Io = I(Do)
    Is = I(Ds)

    Q_rslt = kron(mpo.Q, Is) + kron(Io, mps.Q)
    for i in 1:Do
        for j in 1:Do
            for k in 1:Ds
                for l in 1:Ds
                    Q_rslt[(i-1)*Ds + k, (j-1)*Ds + l] += sum(mpo.R[:, i, j] .* mps.R[:, k, l])
                end
            end
        end
    end

    R_rslt = Array{Float64, 3}(undef, d, Do * Ds, Do * Ds)
    for i in 1:d
        for j in 1:Do
            for k in 1:Ds
                for l in 1:Do
                    for m in 1:Ds
                        R_rslt[i, (j-1)*Ds + k, (l-1)*Ds + m] = mpo.L[i, j, l] * Is[k, m] + sum(mpo.P[i, j, l, :] .* mps.R[:, k, m])
                    end
                end
            end
        end
    end

    return CMPS(Q_rslt, R_rslt)
end

# Usage example
Q = rand(2, 2)
R = rand(2, 2, 2)
mps_instance = CMPS(Q, R)

W = rand(2, 2)
mps_new = multiply(W, mps_instance)

cmpo_Q = rand(2, 2)
cmpo_L = rand(2, 2, 2)
cmpo_R = rand(2, 2, 2)
cmpo_P = rand(2, 2, 2, 2)
cmpo_instance = CMPO(cmpo_Q, cmpo_L, cmpo_R, cmpo_P)

result_mps = act(cmpo_instance, mps_instance)


CMPS([1.5348609915096827 0.9555361563978043 0.18732105555521633 0.4867798792785287; 0.9896056587724039 1.7368396553640035 0.41110873000949744 0.6565966220663343; 1.1760578565608544 0.992247080007989 1.4827070220816878 1.1585721234706559; 0.7805744806107592 2.2697323540480205 1.0326538939588352 2.187047370976318], [0.33357442841110957 0.15677585721682386 1.0330886919290962 0.36267495993737503; 0.8599779084957486 0.774854784718297 1.1577610492961319 0.6975035042502957;;; 0.385309266122349 1.1076619184722802 0.36132974466692075 1.2441034767414667; 0.8776476295706442 1.6257481471778295 0.8564381152704134 2.04499996034813;;; 0.3803574988114949 0.505930665875316 0.3800334027084757 0.775708059905255; 0.3511603159934383 0.6649807180468641 0.9745918777314139 0.17659666015527842;;; 0.6226529847596218 1.028203362016677 0.8182682130948121 0.9667876375780956; 0.7105428858941312 0.8812135101328653 0.30510785654592154 1.4623173235544087], 4)

In [19]:
using LinearAlgebra

function transpose_cmps(cmps::CMPS)
    Q = cmps.Q
    R = permutedims(cmps.R, (1, 3, 2))
    return CMPS(Q, R)
end


function Lact(mps::CMPS, mpo::CMPO)
    Do, Ds = mpo.dim, mps.dim
    d = size(mps.R, 1)

    Tmps = act(transpose_cmpo(mpo), mps)

    Q = reshape(permutedims(reshape(Tmps.Q, Do, Ds, Do, Ds), (2, 1, 4, 3)), Do*Ds, Do*Ds)
    R = reshape(permutedims(reshape(Tmps.R, d, Do, Ds, Do, Ds), (1, 3, 2, 4)), d, Do*Ds, Do*Ds)

    return CMPS(Q, R)
end


function density_matrix(mps1::CMPS, mps2::CMPS)
    D1, D2 = mps1.dim, mps2.dim
    I1 = I(D1)
    I2 = I(D2)

    M = kron(mps1.Q, I2) + kron(I1, mps2.Q)
    for i in 1:size(mps1.R, 1)
        for j in 1:D1
            for k in 1:D1
                for l in 1:D2
                    for m in 1:D2
                        M[(j-1)*D2 + l, (k-1)*D2 + m] += mps1.R[i, j, k] * mps2.R[i, l, m]
                    end
                end
            end
        end
    end

    return M
end


# Usage example
Q1 = rand(2, 2)
R1 = rand(2, 2, 2)
mps1 = CMPS(Q1, R1)

Q2 = rand(2, 2)
R2 = rand(2, 2, 2)
mps2 = CMPS(Q2, R2)

cmpo_Q = rand(2, 2)
cmpo_L = rand(2, 2, 2)
cmpo_R = rand(2, 2, 2)
cmpo_P = rand(2, 2, 2, 2)
cmpo_instance = CMPO(cmpo_Q, cmpo_L, cmpo_R, cmpo_P)

Tmps = Lact(mps1, cmpo_instance)
M = density_matrix(mps1, mps2)



LoadError: ArgumentError: no valid permutation of dimensions

In [22]:
using LinearAlgebra

function ln_ovlp(mps1::CMPS, mps2::CMPS, beta::Float64)
    M = density_matrix(mps1, mps2)
    return logsumexp(beta * eigen(M).values)
end

function Fidelity(psi::CMPS, mps::CMPS, beta::Float64)
    up = ln_ovlp(psi, mps, beta)
    dn = ln_ovlp(psi, psi, beta)
    return up - 0.5 * dn
end

function energy_cut(mps::CMPS, chi::Int)
    w, v = eigen(mps.Q)
    P = v[:, end-chi+1:end]
    return P
end

function interpolate_cut(cut1::Array{Float64, 2}, cut2::Array{Float64, 2}, theta::Float64)
    mix = sin(theta) * cut1 + cos(theta) * cut2
    U, _, V = svd(mix)
    return U * V'
end

# Helper function to calculate logsumexp
function logsumexp(x::Vector{Float64})
    c = maximum(x)
    return c + log(sum(exp.(x .- c)))
end

logsumexp (generic function with 2 methods)

In [24]:
using LinearAlgebra
using Zygote

function adaptive_mera_update(mps::CMPS, beta::Float64, chi::Int, tol::Float64=1e-12, maxiter::Int=50)
    """ update the isometry using iterative SVD update with line search
        mps: the original cMPS
        beta: inverse temperature
        chi: target bond dimension
        return the compressed cMPS
    """
    P = energy_cut(mps, chi)
    last = 9.9e9
    step = 0
    while step < maxiter
        # 定义损失函数
        function loss_function(P)
            P_projected = project(mps, P)
            ln_ovlp_new = ln_ovlp(P_projected, mps, beta)
            ln_ovlp_detached = ln_ovlp(P_projected, deepcopy(mps), beta)
            return ln_ovlp_new - 0.5 * ln_ovlp_detached
        end
        
        # 计算梯度
        loss, back = Zygote.pullback(loss_function, P)
        grad = back(1)[1]
        
        diff = abs(loss - last)
        
        if diff < tol
            break
        end
        
        println(step, "\r")
        
        last = loss
        Fidel0 = loss
        Fidel_test = 1e99
        
        step += 1
        
        U, _, V = svd(grad)
        
        theta = π
        proceed = false
        while !proceed
            theta /= 2
            if theta < π / 1.9^12
                theta = 0
                P_test = P
            else
                P_test = interpolate_cut(U * V', P, theta)
            end
            
            mps_test = project(mps, P_test)
            Fidel1_test = Fidelity(mps_test, mps, beta)
            if Fidel1_test > Fidel0 || isapprox(theta, 0)
                P = P_test
                proceed = true
            end
        end
    end
    
    return project(mps, P)
end

# 使用示例
Q = rand(2, 2)
R = rand(2, 2, 2)
mps_instance = CMPS(Q, R)

beta = 1.0
chi = 1
mps_new = adaptive_mera_update(mps_instance, beta, chi)


LoadError: Mutating arrays is not supported -- called setindex!(Matrix{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [25]:
using LinearAlgebra
using Optim
using JLD2
using Zygote

struct CMPS
    Q::Array{Float64, 2}
    R::Array{Float64, 3}
    dim::Int

    function CMPS(Q, R)
        new(Q, R, size(Q, 1))
    end
end

function adaptive_mera_update(mps::CMPS, beta::Float64, chi::Int, tol::Float64=1e-12, maxiter::Int=50)
    P = energy_cut(mps, chi)
    last = 9.9e9
    step = 0
    while step < maxiter
        function loss_function(P)
            P_projected = project(mps, P)
            ln_ovlp_new = ln_ovlp(P_projected, mps, beta)
            ln_ovlp_detached = ln_ovlp(P_projected, deepcopy(mps), beta)
            return ln_ovlp_new - 0.5 * ln_ovlp_detached
        end
        
        loss, back = Zygote.pullback(loss_function, P)
        grad = back(1)[1]
        
        diff = abs(loss - last)
        
        if diff < tol
            break
        end
        
        println(step, "\r")
        
        last = loss
        Fidel0 = loss
        Fidel_test = 1e99
        
        step += 1
        
        U, _, V = svd(grad)
        
        theta = π
        proceed = false
        while !proceed
            theta /= 2
            if theta < π / 1.9^12
                theta = 0
                P_test = P
            else
                P_test = interpolate_cut(U * V', P, theta)
            end
            
            mps_test = project(mps, P_test)
            Fidel1_test = Fidelity(mps_test, mps, beta)
            if Fidel1_test > Fidel0 || isapprox(theta, 0)
                P = P_test
                proceed = true
            end
        end
    end
    
    return project(mps, P)
end

function variational_compr(mps::CMPS, beta::Float64, chi::Int, chkp_loc::String, init::Union{CMPS, Nothing}=nothing, tol::Float64=1e-12)
    if init === nothing
        psi = adaptive_mera_update(mps, beta, chi, tol=tol)
        psi = diagQ(psi)
    else
        psi = init
    end

    Q = diagm(psi.Q)
    R = psi.R

    function loss_function(vars)
        Q, R = reshape(vars[1:chi], (chi, chi)), reshape(vars[chi+1:end], size(psi.R))
        psi = CMPS(Q, R)
        return -Fidelity(psi, mps, beta)
    end

    function gradient(vars)
        Q, R = reshape(vars[1:chi], (chi, chi)), reshape(vars[chi+1:end], size(psi.R))
        psi = CMPS(Q, R)
        loss, back = Zygote.pullback(() -> -Fidelity(psi, mps, beta), vars)
        return back(1)[1]
    end

    vars = vcat(vec(Q), vec(R))
    result = optimize(loss_function, gradient, vars, LBFGS(); g_tol=tol, f_tol=tol, maxIter=20)
    
    vars_opt = Optim.minimizer(result)
    Q_opt, R_opt = reshape(vars_opt[1:chi], (chi, chi)), reshape(vars_opt[chi+1:end], size(psi.R))

    # 归一化
    Q_opt .-= maximum(Q_opt)
    psi = CMPS(Q_opt, R_opt)

    # 保存检查点
    JLD2.@save chkp_loc psi

    return psi
end

# 帮助函数
function diagQ(cmps::CMPS)
    F = eigen(cmps.Q)
    U = F.vectors
    return project(cmps, U)
end

# 使用示例
Q = rand(2, 2)
R = rand(2, 2, 2)
mps_instance = CMPS(Q, R)

beta = 1.0
chi = 1
chkp_loc = "checkpoint.jld2"

mps_new = variational_compr(mps_instance, beta, chi, chkp_loc)


LoadError: ArgumentError: Package Optim not found in current path.
- Run `import Pkg; Pkg.add("Optim")` to install the Optim package.